In [1]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pickle as pkl

# load tokenized dataset

In [2]:
dataset_dir_path = '../../data/processed/tokenized_data/'
with open(dataset_dir_path + 'train_data.pkl', 'rb') as f:
    tokenized_train_data = pkl.load(f)

with open(dataset_dir_path + 'valid_data.pkl', 'rb') as f:
    tokenized_valid_data = pkl.load(f)

In [3]:
print(f'Input(de) {tokenized_train_data[0][0]}')
print(f'Output(en) {tokenized_train_data[0][1]}')

Input(de) tensor([   2,   21,   85,  256,   31,   86,   22,   93,    7,   16,  114, 5645,
        3245,    3])
Output(en) tensor([   2,   19,   25,   15, 1197,  817,   17,   58,   84,  332, 1319,    3])


# load vocab

In [4]:
vocab_dir_path = '../../data/processed/vocab/'

with open(vocab_dir_path + 'token2idx_de.pkl', 'rb') as f:
    token2idx_de= pkl.load(f)
with open(vocab_dir_path + 'token2idx_en.pkl', 'rb') as f:
    token2idx_en = pkl.load(f)
with open(vocab_dir_path + 'idx2token_de.pkl', 'rb') as f:
    idx2token_de = pkl.load(f)
with open(vocab_dir_path + 'idx2token_en.pkl', 'rb') as f:
    idx2token_en = pkl.load(f)

# making the batch

In [5]:
batch_size = 128
PAD_INDEX = token2idx_de['<pad>']
START_INDEX = token2idx_en['<start>']
END_INDEX = token2idx_en['<end>']

In [6]:
def generate_batch(data_batch):
    batch_src = []
    batch_tgt = []
    for src, tgt in data_batch:
        batch_src.append(src)
        batch_tgt.append(tgt)
    
    batch_src = pad_sequence(batch_src, padding_value=PAD_INDEX)
    batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_INDEX)

    return batch_src, batch_tgt

In [7]:
train_iter = DataLoader(tokenized_train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(tokenized_valid_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)

In [8]:
# show train_iter
# each column is a text
list(train_iter)[0]

(tensor([[   2,    2,    2,  ...,    2,    2,    2],
         [   5,   21,  265,  ...,    5,   21,   21],
         [1658,   85, 3228,  ...,   27,   43,  115],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]]),
 tensor([[  2,   2,   2,  ...,   2,   2,   2],
         [  6,  19, 220,  ...,   6,  19,  19],
         [533,  25,  10,  ...,  33,  22,  25],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]))

In [9]:
for src, tgt in train_iter:
    print(src)
    print(tgt)
    print(f'src shape : {src.shape}')
    print(f'tgt.shape : {tgt.shape}')
    break

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  21,   87,  126,  ...,    5,  722,    5],
        [  31,  256, 3179,  ...,  949,    7, 2244],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  19,   49,   49,  ...,   53, 2675,    6],
        [  36,   24,  403,  ...,  141,  226, 1710],
        ...,
        [   1,    1,    1,  ...,    1,    4,    1],
        [   1,    1,    1,  ...,    1,  767,    1],
        [   1,    1,    1,  ...,    1,    3,    1]])
src shape : torch.Size([24, 128])
tgt.shape : torch.Size([27, 128])


# Model

In [10]:
import math
import torch
import torch.nn as nn
from torch import Tensor

## token embedding

In [11]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        # nn.Embedding is a simple lookup table.
        # if token index is set, it will return the corresponding embedding vector.
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding_size = embedding_size
    
    def forward(self, tokens: Tensor):
        # the reason for this multiply is to align the range of the values
        # It is to make the positional encoding relatively smaller. 
        # This means the original meaning in the embedding vector won’t be lost 
        # when we add them together.
        return self.embedding(tokens.long()) * math.sqrt(self.embedding_size)

## positional encoging

In [ ]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, embedding_size: int, dropout: float, maxlen: int = 5000):
        super().__init__()
        
        den = torch.exp(-torch.arange(0, embedding_size, 2) * math.log(10000) / embedding_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        embedding_pos = torch.zeros((maxlen, embedding_size))
        embedding_pos[:, 0::2] = torch.sin(pos * den) # extract even element (start:stop:step)
        embedding_pos[:, 1::2] = torch.cos(pos * den) # extract odd element (start:stop:step)
        embedding_pos = embedding_pos.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('embedding_pos', embedding_pos) # positional encoding is not updated by learning

    def forward(self, token_embedding: Tensor):
        #print(f'token_embedding : {token_embedding.shape}')
        #print(f'positional_encoding : {self.embedding_pos.shape}') # self.embedding_pos is defined by self.register_fuffer.
        # self.embedding_pos is not updated by learning
        #print(f'positional_encoding : {self.embedding_pos[:token_embedding.size(0), :].shape}')
        return self.dropout(token_embedding + self.embedding_pos[:token_embedding.size(0), :])

## masking

In [13]:
def generate_square_subsequent_mask(seq_len, PAD_INDEX):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # seq_len is the length of the sentence
    # The mask is used to prevent the model from looking ahead in the sequence.
    # The mask is a square matrix of size (seq_len, seq_len)
    # The upper triangle of the matrix is filled with -inf
    # The lower triangle of the matrix is filled with 0
    mask = (torch.triu(torch.ones((seq_len, seq_len), device=device)) == 1).transpose(0, 1) # upper triangle
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == PAD_INDEX, float(0.0)) # 1 -> -inf, 0 -> 0
    return mask

def create_mask(src, tgt, PAD_INDEX):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    seq_len_src = src.shape[0] # word number in one sentence in source
    seq_len_tgt = tgt.shape[0] # word number in one sentence in target

    mask_tgt = generate_square_subsequent_mask(seq_len_tgt, PAD_INDEX=PAD_INDEX)
    mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

    padding_mask_src = (src == PAD_INDEX).transpose(0, 1)
    padding_mask_tgt = (tgt == PAD_INDEX).transpose(0, 1)

    return mask_src, mask_tgt, padding_mask_src, padding_mask_tgt

In [14]:
seq_len_src = src.shape[0]
mask_src = torch.zeros((seq_len_src, seq_len_src)).type(torch.bool)

In [15]:
mask_src

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, Fals

In [16]:
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer

In [17]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int, embedding_size: int, vocab_size_src: int, vocab_size_tgt: int, dim_feedforward: int = 512, dropout: float = 0.1, nhead:int = 8):
        super().__init__()

        self.token_embedding_src = TokenEmbedding(vocab_size_src, embedding_size)
        self.positional_encoding = PositionalEncoding(embedding_size, dropout=dropout)
        encoder_layer = TransformerEncoderLayer(d_model=embedding_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_encoder_layers)

        self.token_embedding_tgt = TokenEmbedding(vocab_size_tgt, embedding_size)
        decoder_layer = TransformerDecoderLayer(d_model=embedding_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layer=decoder_layer, num_layers=num_decoder_layers)
        self.output = nn.Linear(embedding_size, vocab_size_tgt)
    
    def forward(self, src: Tensor, tgt: Tensor, mask_src: Tensor, mask_tgt: Tensor, padding_mask_src: Tensor, padding_mask_tgt: Tensor, memory_key_padding_mask: Tensor):
        # src : (seq_len_src, batch_size)
        # tgt : (seq_len_tgt, batch_size)
        # mask_src : (seq_len_src, seq_len_src)
        # mask_tgt : (seq_len_tgt, seq_len_tgt)
        # padding_mask_src : (batch_size, seq_len_src)
        # padding_mask_tgt : (batch_size, seq_len_tgt)

        embedding_src = self.positional_encoding(self.token_embedding_src(src))
        memory = self.transformer_encoder(embedding_src, mask_src, padding_mask_src)
        embedding_tgt = self.positional_encoding(self.token_embedding_tgt(tgt))
        outs = self.transformer_decoder(embedding_tgt, memory, mask_tgt, None, padding_mask_tgt, memory_key_padding_mask)
        return self.output(outs)
    
    def encode(self, src: Tensor, mask_src: Tensor):
        return self.transformer_encoder(self.positional_encoding(self.token_embedding_src(src)), mask_src)
    
    def decode(self, tgt: Tensor, memory: Tensor, mask_tgt: Tensor):
        return self.transformer_decoder(self.positional_encoding(self.token_embedding_tgt(tgt), memory, mask_tgt))


In [18]:
from tqdm import tqdm

In [19]:
def train(model, data, optimizer, criterion, PAD_INDEX):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.train()
    losses = 0.0
    for src, tgt in tqdm(data):
        src = src.to(device)
        tgt = tgt.to(device)
        input_tgt = tgt[:-1, :] # remove last token
        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_INDEX)

        logits = model(src=src, tgt=input_tgt,
                        mask_src=mask_src, mask_tgt=mask_tgt,
                        padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
                        memory_key_padding_mask=padding_mask_src)

        optimizer.zero_grad()

        output_tgt = tgt[1:, :] # remove first token
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    
    return losses / len(data)

In [20]:
def evaluate(model, data, criterion, PAD_INDEX):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.eval()
    losses = 0.0

    for src, tgt in data:
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :] # remove last token
        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_INDEX)
        logits = model(src=src, tgt=input_tgt, 
                       mask_src=mask_src, mask_tgt=mask_tgt,
                       padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
                       memory_key_padding_mask=padding_mask_src)

        output_tgt = tgt[1:, :] # remove first token
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        losses += loss.item()

    return losses / len(data)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_size_src = len(token2idx_de)
vocab_size_tgt = len(token2idx_en)
embedding_size = 240 # smaller than original 512
nhead = 8
dim_feedforward = 100
num_encoder_layers = 2
num_decoder_layers = 2
dropout = 0.1

model = Seq2SeqTransformer(
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    embedding_size=embedding_size,
    vocab_size_src=vocab_size_src,
    vocab_size_tgt=vocab_size_tgt,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    nhead=nhead
)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p) # Xavier initialization

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX)
optimizer = torch.optim.Adam(model.parameters())

/home/sasatake/repos/python/transformer/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [22]:
import time

epoch = 100
best_loss = float('inf')
best_model = None
patience = 10
counter = 0

for loop in range(epoch):
    start_time = time.time()

    loss_train = train(
        model=model, data=train_iter, optimizer=optimizer,
        criterion=criterion, PAD_INDEX=PAD_INDEX
    )

    elapsed_time = time.time() - start_time
    loss_valid = evaluate(
        model=model, data=valid_iter, criterion=criterion, PAD_INDEX=PAD_INDEX
    )

    print(f'epoch: {loop+1}, train loss: {loss_train:.4f}, valid loss: {loss_valid:.4f}, elapsed time: {elapsed_time:.4f} sec')

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = model
        counter = 0
    
    if counter > patience:
        break

    counter += 1

  0%|          | 0/227 [00:00<?, ?it/s]/home/sasatake/repos/python/transformer/.venv/lib/python3.10/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


token_embedding : torch.Size([26, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([26, 1, 240])
token_embedding : torch.Size([24, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([24, 1, 240])


  2%|▏         | 5/227 [00:00<00:15, 14.62it/s]

token_embedding : torch.Size([28, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([28, 1, 240])
token_embedding : torch.Size([26, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([26, 1, 240])
token_embedding : torch.Size([37, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([37, 1, 240])
token_embedding : torch.Size([38, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([38, 1, 240])
token_embedding : torch.Size([35, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([35, 1, 240])
token_embedding : torch.Size([27, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([27, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embeddi

  6%|▌         | 13/227 [00:00<00:07, 27.03it/s]

...

 96%|█████████▌| 217/227 [00:05<00:00, 38.50it/s]

token_embedding : torch.Size([25, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([25, 1, 240])
token_embedding : torch.Size([26, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([26, 1, 240])
token_embedding : torch.Size([26, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([26, 1, 240])
token_embedding : torch.Size([31, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([31, 1, 240])
token_embedding : torch.Size([32, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([32, 1, 240])
token_embedding : torch.Size([32, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([32, 1, 240])
token_embedding : torch.Size([27, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([27, 1, 240])
token_embeddi

100%|██████████| 227/227 [00:05<00:00, 38.47it/s]

token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embedding : torch.Size([30, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([30, 1, 240])
token_embedding : torch.Size([27, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([27, 1, 240])
token_embedding : torch.Size([24, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([24, 1, 240])
token_embedding : torch.Size([27, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([27, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embeddi

token_embedding : torch.Size([30, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([30, 1, 240])
token_embedding : torch.Size([30, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([30, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embedding : torch.Size([32, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([32, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embedding : torch.Size([29, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([29, 1, 240])
token_embedding : torch.Size([31, 128, 240])
positional_encoding : torch.Size([5000, 1, 240])
positional_encoding : torch.Size([31, 1, 240])
token_embeddi